# Pareto Front Visualization

This notebook visualizes the Pareto fronts for different k values.

Features:
- 3D scatter plots of Pareto fronts (difficulty vs diversity vs balance)
- Convergence curves over generations
- Comparison of Pareto fronts across different k values


In [ ]:
import sys
from pathlib import Path
import pickle
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

# Add parent directory to path
sys.path.insert(0, str(Path().absolute().parent))
import config

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)


In [ ]:
# Load Pareto fronts for all k values
pareto_data = {}
for k in config.K_VALUES:
    pareto_path = config.get_pareto_front_path(k)
    if pareto_path.exists():
        with open(pareto_path, 'rb') as f:
            result = pickle.load(f)
            pareto_data[k] = {
                'fitnesses': result['pareto_front']['fitnesses'],
                'history': result['history']
            }
        print(f"Loaded k={k}: {len(result['pareto_front']['fitnesses'])} Pareto-optimal solutions")
    else:
        print(f"Warning: Pareto front for k={k} not found at {pareto_path}")


## 3D Pareto Front Visualization


In [ ]:
# Create 3D scatter plots for each k
fig = plt.figure(figsize=(18, 12))

for idx, k in enumerate(config.K_VALUES, 1):
    if k not in pareto_data:
        continue
    
    ax = fig.add_subplot(2, 3, idx, projection='3d')
    
    fitnesses = np.array(pareto_data[k]['fitnesses'])
    difficulties = fitnesses[:, 0]
    diversities = fitnesses[:, 1]
    balances = fitnesses[:, 2]
    
    scatter = ax.scatter(difficulties, diversities, balances, 
                         c=range(len(fitnesses)), cmap='viridis', 
                         s=50, alpha=0.6, edgecolors='black', linewidth=0.5)
    
    ax.set_xlabel('Difficulty', fontsize=10)
    ax.set_ylabel('Diversity', fontsize=10)
    ax.set_zlabel('Balance', fontsize=10)
    ax.set_title(f'Pareto Front: k={k}\n({len(fitnesses)} solutions)', fontsize=12, fontweight='bold')
    
    # Add colorbar
    plt.colorbar(scatter, ax=ax, label='Solution Index', shrink=0.8)

plt.tight_layout()
plt.savefig('results/pareto_fronts_3d.png', dpi=150, bbox_inches='tight')
plt.show()


## Convergence Over Generations


In [ ]:
# Plot convergence for each k
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

for idx, k in enumerate(config.K_VALUES):
    if k not in pareto_data:
        continue
    
    ax = axes[idx]
    history = pareto_data[k]['history']
    
    generations = history['generation']
    pareto_sizes = history['pareto_front_size']
    best_difficulty = history['best_difficulty']
    best_diversity = history['best_diversity']
    best_balance = history['best_balance']
    
    # Plot Pareto front size
    ax2 = ax.twinx()
    line1 = ax.plot(generations, best_difficulty, 'b-', label='Best Difficulty', linewidth=2)
    line2 = ax.plot(generations, best_diversity, 'g-', label='Best Diversity', linewidth=2)
    line3 = ax.plot(generations, best_balance, 'r-', label='Best Balance', linewidth=2)
    line4 = ax2.plot(generations, pareto_sizes, 'm--', label='Pareto Front Size', linewidth=2, alpha=0.7)
    
    ax.set_xlabel('Generation', fontsize=10)
    ax.set_ylabel('Best Objective Value', fontsize=10, color='black')
    ax2.set_ylabel('Pareto Front Size', fontsize=10, color='m')
    ax.set_title(f'Convergence: k={k}', fontsize=12, fontweight='bold')
    ax.grid(True, alpha=0.3)
    
    # Combine legends
    lines = line1 + line2 + line3 + line4
    labels = [l.get_label() for l in lines]
    ax.legend(lines, labels, loc='best', fontsize=8)

plt.tight_layout()
plt.savefig('results/convergence_curves.png', dpi=150, bbox_inches='tight')
plt.show()


## Compare Pareto Fronts Across k Values


In [ ]:
# Compare Pareto front sizes and best objectives across k values
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Pareto front sizes
ax = axes[0, 0]
k_values = [k for k in config.K_VALUES if k in pareto_data]
pareto_sizes = [len(pareto_data[k]['fitnesses']) for k in k_values]
ax.bar(k_values, pareto_sizes, color='steelblue', alpha=0.7, edgecolor='black')
ax.set_xlabel('Subset Size (k)', fontsize=11)
ax.set_ylabel('Pareto Front Size', fontsize=11)
ax.set_title('Pareto Front Size vs k', fontsize=12, fontweight='bold')
ax.grid(axis='y', alpha=0.3)

# Best difficulty
ax = axes[0, 1]
best_difficulties = [max(pareto_data[k]['history']['best_difficulty']) for k in k_values]
ax.plot(k_values, best_difficulties, 'o-', linewidth=2, markersize=8, color='blue')
ax.set_xlabel('Subset Size (k)', fontsize=11)
ax.set_ylabel('Best Difficulty', fontsize=11)
ax.set_title('Best Difficulty vs k', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)

# Best diversity
ax = axes[1, 0]
best_diversities = [max(pareto_data[k]['history']['best_diversity']) for k in k_values]
ax.plot(k_values, best_diversities, 'o-', linewidth=2, markersize=8, color='green')
ax.set_xlabel('Subset Size (k)', fontsize=11)
ax.set_ylabel('Best Diversity', fontsize=11)
ax.set_title('Best Diversity vs k', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)

# Best balance
ax = axes[1, 1]
best_balances = [max(pareto_data[k]['history']['best_balance']) for k in k_values]
ax.plot(k_values, best_balances, 'o-', linewidth=2, markersize=8, color='red')
ax.set_xlabel('Subset Size (k)', fontsize=11)
ax.set_ylabel('Best Balance', fontsize=11)
ax.set_title('Best Balance vs k', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('results/pareto_comparison.png', dpi=150, bbox_inches='tight')
plt.show()
